# Resources

Brilliant talk by Ben Moseley on PINN:
https://www.youtube.com/watch?v=G_hIppUWcsc

# Install necessary pip modules/packages

Using good ipynb practise:
https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

Install the necessary pip packages and modules using pip in the virtual environment (ensure README.md has been followed):

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

Import the necessary packages and modules:

In [31]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

Check that cuda and gpu is available.

In [ ]:
torch.cuda.is_available()
torch.cuda.device_count()

# Analytic solution to damped harmonic oscillator

In [33]:
def exact_solution_underdamped(d, w0, t):
    assert d < w0  # ensure under-damped case (gives oscillatory motion)

    w = np.sqrt(w0**2 - d**2)
    phi = np.arctan(-d/w)  # phase
    
    exp_term = np.exp(-d*t)
    A = 1.0/(2.0*np.cos(phi))
    cos_term = np.cos(phi + w*t)

    u = exp_term * 2.0 * A * cos_term
    return u

Assume m=1 then:

d = mu/(2*m)    =>  mu = 2d
w0 = sqrt(k/m)  =>  k = w0**2

Plot some of the exact solutions to get a feel for the solution.

In [45]:
t_test1 = np.linspace(0, 1, 300)

# Control case: d, w0 = 2, 20
d, w0 = 2, 20
mu, k = 2.0*d, w0**2
u_exact_d_2_w0_20 = exact_solution_underdamped(d, w0, t_test1)

# Higher damping (higher d):  d, w0 = 4, 20
d, w0 = 4, 20
mu, k = 2.0*d, w0**2
u_exact_d_4_w0_20 = exact_solution_underdamped(d, w0, t_test1)

# Higher spring constant (higher w0):  
d, w0 = 2, 40
mu, k = 2.0*d, w0**2
u_exact_d_2_w0_40 = exact_solution_underdamped(d, w0, t_test1)

In [ ]:
plt.scatter(t_test1, u_exact_d_2_w0_20, label="d=2 w0=20 (Control)")
plt.scatter(t_test1, u_exact_d_4_w0_20, label="d=4 w0=20 (High damping)")
plt.scatter(t_test1, u_exact_d_2_w0_40, label="d=2 w0=40 (Higher spring constant/frequency)")
plt.legend()
plt.show()

# Simple fully connected neural network

Define a simple, fully connected neural network with variable number of hidden layers.  This NN will be trained on the analytic results using the physics-informed loss function (not defined yet) which is composed of 2 loss terms: L_data and L_model.

In [10]:
class fully_connected_network(nn.Module):
    """
    Define a simple fully-connected network in Pytorch with takes a 1D tensor as input and outputs a 1D tensor as output.
    
    Arguments:
    - n_input  : size of the input 1D tensor
    - n_output : size of the output 1D tensor
    - n_hidden : size of the 1D hidden layers
    - n_hidden_layers : number of hidden layers
    """
    
    def __init__(self, n_input, n_output, n_hidden, n_hidden_layers):
        super(fully_connected_network, self).__init__()
        
        activation = nn.Tanh
        
        self.fc_in = nn.Sequential(
            *[
                nn.Linear(n_input, n_hidden),
                activation()
            ]
        )

        self.fc_hidden = nn.Sequential(
            *[
                nn.Sequential(*[nn.Linear(n_hidden, n_hidden), activation()]) for _ in range(0, n_hidden_layers-1)
            ]
        )

        self.fc_out = nn.Linear(n_hidden, n_output)
    
    def forward(self, x):
        x = self.fc_in(x)
        x = self.fc_hidden(x)
        x = self.fc_out(x)

        return x

Test the network with random data.

In [ ]:
print(fully_connected_network)

for i in range(0, 10000):
    random_data = torch.rand((5))
    print("random_data: ", random_data)

    test_nn_1 = fully_connected_network(5, 2, 10, 10)
    result = test_nn_1(random_data)
    print("result: ", result)

In [ ]:
torch.manual_seed(123)